### Bibliothèque

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor


### Import

In [ ]:
# Charger le jeu de données dans un DataFrame pandas
df = pd.read_csv('data_film_model.csv')

In [ ]:
pd.options.display.max_columns = 150

In [ ]:
df.columns

In [ ]:
df.head(1)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
print(df.duplicated().value_counts())

### Train/Test/Split

In [ ]:
X = df.drop(['Unnamed: 0', 'title', 'budget',  'press_eval', 'viewers_eval','year','stars_producers_director','box_office_first_week'],axis=1)
y = df['box_office_first_week']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

In [ ]:
print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
X.isnull().sum()

In [ ]:
X.head()

### Pipeline

In [ ]:
# Définir les paramètres du préprocesseur
numerical_features = ['duration_minutes',  'nombre_acteurs_connus']
categorical_features = ['nationality', 'season']

In [ ]:
# Création de pipelines
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()


### Preprocessing

In [ ]:
# Création d'un Préprocessor pour appliquer les pipelines crées pour les transfomations sur les colonnes
preprocessing = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder="passthrough"
)

"""
Si pour un type de variable, on a pas de transformer, il faut utiliser
remainder="passthrough"      
"""


### Régression Linéaire

In [ ]:
# Modèle de Regression Linéaire


model_lr = make_pipeline(preprocessing, LinearRegression())

model_lr.fit(X_train,y_train)


In [ ]:
# Accéder à l'étape de mise à l'échelle à l'aide de named_steps
linear = model_lr[-1]

In [ ]:
print(type(linear))

In [ ]:

# Prédictions sur les ensembles d'entraînement et de test
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

# Calculer R² et MAE pour l'ensemble d'entraînement
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculer R² et MAE pour l'ensemble de test
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Afficher les résultats
print(f'R² (Entraînement): {r2_train:.2f}')
print(f'MAE (Entraînement): {mae_train:.2f}')
print(f'R² (Test): {r2_test:.2f}')
print(f'MAE (Test): {mae_test:.2f}')

In [ ]:
model_lr.score(X_train, y_train)

In [ ]:
y_pred = model_lr.predict(X_train)

In [ ]:
# Calcul du R²
r2 = r2_score(y_train, y_pred)
print("R² : %.2f" % r2)

In [ ]:
# Calcul du MAE
mae = mean_absolute_error(y_train, y_pred)
print("MAE : %.2f" % mae)

In [ ]:
y_pred = model_lr.predict(X_train)

In [ ]:
# Faire des prédictions sur les données de test
y_pred = model_lr.predict(X_test)

y_pred_train = model_lr.predict(X_train)

In [ ]:
print("R-squared (R2) : ")
print("TRAIN :", r2_score(y_train, y_pred_train))
print("TEST :", r2_score(y_test, y_pred))

### XGBoost

In [ ]:
# Définition de la grille de paramètres
params = {
    'n_estimators': 200,
    'max_depth': 3,
    'learning_rate': 0.2
}

In [ ]:
xgbr = make_pipeline(preprocessing, XGBRegressor())

xgbr.fit(X_train, y_train)

In [ ]:
xgbr.get_params

In [ ]:
xgbr.score(X_train, y_train)*100

In [ ]:
xgbr.score(X_test, y_test)*100

In [ ]:
y_pred_2 = xgbr.predict(X_train)

In [ ]:
y_pred_3 = xgbr.predict(X_test)

In [ ]:
# Calcul du R²
r2 = r2_score(y_train, y_pred_2)
print("R² : %.2f" % r2)

In [ ]:
# Calcul du R²
r2 = r2_score(y_test, y_pred_3)
print("R² : %.2f" % r2)

In [ ]:
# Calcul du MAE
mae = mean_absolute_error(y_train, y_pred_2)
print("MAE : %.2f" % mae)

In [ ]:

# Prédictions sur les ensembles d'entraînement et de test
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

# Calculer R² et MAE pour l'ensemble d'entraînement
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculer R² et MAE pour l'ensemble de test
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Afficher les résultats
print(f'R² (Entraînement): {r2_train:.2f}')
print(f'MAE (Entraînement): {mae_train:.2f}')
print(f'R² (Test): {r2_test:.2f}')
print(f'MAE (Test): {mae_test:.2f}')

### GridSearch

In [ ]:

# Définir les caractéristiques numériques et catégorielles
numerical_features = ['duration_minutes', 'nombre_acteurs_connus']
categorical_features = ['nationality', 'season']

# Définir les transformateurs
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Créer le préprocesseur avec ColumnTransformer
preprocessing = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder="passthrough"
)

# Créer le pipeline avec XGBoost
xgb_model = Pipeline([
    ('preprocessing', preprocessing),
    ('xgb', XGBRegressor())
])

# Définir la grille de recherche des hyperparamètres
param_grid = {
    'xgb__n_estimators': [50, 100, 200],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__max_depth': [3, 5, 7]
}

# Effectuer la recherche des meilleurs paramètres avec GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1)

# Adapter le modèle
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres pour XGBoost:", grid_search.best_params_)

# Obtenir les meilleures estimations
best_model = grid_search.best_estimator_

# Prédictions sur les ensembles d'entraînement et de test
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculer R² et MAE pour l'ensemble d'entraînement
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculer R² et MAE pour l'ensemble de test
r2_test = r2_score(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Afficher les résultats
print(f'R² (Entraînement): {r2_train:.2f}')
print(f'MAE (Entraînement): {mae_train:.2f}')
print(f'R² (Test): {r2_test:.2f}')
print(f'MAE (Test): {mae_test:.2f}')

In [ ]:
grid_search.score(X_train, y_train)*100

In [ ]:
grid_search.score(X_test, y_test)*100

### Courbe de Validation

In [ ]:
# Courbe de validation pour XGBoost
param_range = np.arange(50, 250, 25)
train_scores, test_scores = validation_curve(
    grid_search.best_estimator_, X_train, y_train, param_name="xgb__n_estimators", param_range=param_range,
    scoring="r2", cv=5
)

plt.figure(figsize=(10, 6))
plt.plot(param_range, np.mean(train_scores, axis=1), label="Entraînement")
plt.plot(param_range, np.mean(test_scores, axis=1), label="Validation")
plt.title("Courbe de validation pour XGBoost")
plt.xlabel("Nombre d'estimateurs")
plt.ylabel("Score R2")
plt.legend()
plt.show()
print(X_train.shape)
print(X_test.shape)


### Courbe d'apprentissage

In [ ]:
# Courbe d'apprentissage pour XGBoost

# Obtenir le meilleur estimateur de la recherche de grille
best_estimator = grid_search.best_estimator_

# Tracer la courbe d'apprentissage
train_sizes, train_scores, test_scores = learning_curve(
    best_estimator, X_train, y_train, scoring="r2", cv=5
)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label="Entraînement")
plt.plot(train_sizes, np.mean(test_scores, axis=1), label="Validation")
plt.title("Courbe d'apprentissage pour XGBoost")
plt.xlabel("Taille de l'ensemble d'entraînement")
plt.ylabel("Score R2")
plt.legend()
plt.show()
print(X_train.shape)
print(X_test.shape)



### Score Final

In [ ]:
# Prédiction sur l'ensemble de test
y_pred = grid_search.predict(X_test)

# Évaluation des performances du modèle
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)*100
mae = mean_absolute_error(y_test, y_pred)

print("R2 Score sur l'ensemble de test:", r2)
print("Erreur absolue moyenne (MAE) sur l'ensemble de test:  %.2f" % mae)


### Feature Importance

In [ ]:
# Réappliquer le prétraitement aux données d'entraînement
X_train_preprocessed = grid_search.best_estimator_.named_steps['preprocessing'].transform(X_train)

# Obtenir les caractéristiques utilisées lors de l'apprentissage du modèle
features = grid_search.best_estimator_.named_steps['preprocessing'].get_feature_names_out()

# Obtenir les importances des caractéristiques à partir du meilleur estimateur
feature_importances = grid_search.best_estimator_.named_steps['xgb'].feature_importances_

# Créer un DataFrame pour afficher les importances des caractéristiques
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Sélectionner les 15 caractéristiques les plus importantes
top_15_features = importance_df.nlargest(15, 'Importance')

# Tracer les importances des caractéristiques
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=top_15_features)
plt.title("Importance des 15 caractéristiques les plus importantes dans le modèle XGBoost")
plt.xlabel("Importance")
plt.ylabel("Caractéristiques")
plt.show()

In [ ]:
grid_search

In [ ]:
import joblib

# Enregistrement du modèle sous format pickle
joblib.dump(grid_search, 'model.pkl')

In [ ]:
test = X.iloc[0]
test

In [ ]:
X.columns

In [ ]:
X.dtypes

In [ ]:
X.columns.shape